# 1. Import of input data

In [1]:
import nbimporter
import sys
import os
from pathlib import Path

In [2]:
#Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 CombinedHeatPower_InternalCombustionEngine.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 CombinedHeatPower_InternalCombustionEngine.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# Se non trovato, prova nei livelli superiori
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Esegui il file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

📁 CombinedHeatPower_InternalCombustionEngine.ipynb executed standalone — using directory: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\LCA4DHC\Energy_systems
📘 Importing libraries from: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\LCA4DHC\Libraries.ipynb
📁 Libraries.ipynb — directory impostata a: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\LCA4DHC


# 2. Input parameters

In [3]:
energysystem = pd.read_excel(energysystem_path, sheet_name = 'CHP-Internal Combustion Engine')

In [4]:
Power = energysystem['Thermal capacity [kW]'];
EfficiencyW = energysystem['Electrical efficiency based on LHV (between 0 and 1)']
EfficiencyQ = energysystem['Heat efficiency based on LHV (between 0 and 1)']
ElectricEnergyproduced = energysystem['Electric energy produced in a year (exported and consumed) [kWh /year]']
ThermalEnergyproduced_primary = energysystem['Thermal energy produced used [kWh /year]']
SupplyTemperature = energysystem['Supply temperature [°C]']
ReturnTemperature = energysystem['Return temperature [°C]']
EnergyInputSourceGasBlend = energysystem['Energy input from gas-blend [kWh/year]']
EnergyInputSourceBiogas = energysystem['Energy input from biogas [kWh/year]']
EnergyInputSourceHydrogen = energysystem['Energy input from hydrogen [kWh/year]']

EquivalentHours = ThermalEnergyproduced_primary / Power

In [5]:
# DEFAULT PARAMETERS
LHV = 36.8  # In MJ per Nm3 - Table 3.6 06_V_Natural Gas 3_3 Fuel Data
HHV = 40.4  # In MJ per Nm3 - Table 3.6 06_V_Natural Gas 3_3 Fuel Data

Lifespan = np.where(
    Power * (EfficiencyW / EfficiencyQ)  < 25,
    40000 / EquivalentHours,
    100000 / EquivalentHours
)

# ADDITIONAL CALCULATIONS
Qused = 1
EfficiencyTot = EfficiencyW + EfficiencyQ
EnergyProducedLHV = (Power * EquivalentHours * RSP) / EfficiencyW
EnergyProducedHHV = EnergyProducedLHV * (HHV / LHV)
TotalW = EnergyProducedLHV * EfficiencyW
TotalQ = EnergyProducedLHV * EfficiencyQ * Qused

In [6]:
# MULTIFUNCTIONALITY AND ALLOCATION OF output
if Multifunctionality == 'Default': 
    ExternalTemperature = ExternalTemperature + 273.15 #K
    SupplyTemperature = SupplyTemperature + 273.15 #K
    ReturnTemperature = ReturnTemperature + 273.15 #K
    CarnotTemperature = (SupplyTemperature - ReturnTemperature) / np.log(SupplyTemperature / ReturnTemperature)
    ThermalEnergyproduced = EnergyProducedHHV
    ElectricEnergyproduced = (EnergyProducedHHV / EfficiencyQ) * EfficiencyW
    # AFQ calculation
    AFQ = (ThermalEnergyproduced * (1 - (ExternalTemperature / CarnotTemperature))) / (ElectricEnergyproduced + (ThermalEnergyproduced * (1 - (ExternalTemperature / CarnotTemperature))))

    AFQ_RICE = np.array(AFQ)
    # AFW calculation
    AFW_RICE = (1 - AFQ_RICE)
     
else:
    fwecr = 0.7
    fweexp = fwecr
    
    # Calculation
    InletEnergyCogenmode = EnergyProducedHHV * EfficiencyTot
    AFW = (InletEnergyCogenmode * fwecr - (ElectricEnergyproduced * fweexp)) / EnergyProducedHHV
    AFW_RICE = np.array([AFW]).T
    AFQ_RICE = 1 - AFW_RICE

if SupplyTemperature.eq(0).all() and ReturnTemperature.eq(0).all():
    AFQ_RICE = np.zeros_like(AFQ_RICE)
    AFW_RICE = np.zeros_like(AFW_RICE)

# 3. Life cycle inventories

## 3.1 Module A1-3

In [7]:
if Power.all(0) < 25:
    mass = 12783.67
    RICE_Air = np.full(len(Power),0) * AFQ_RICE
    RICE_Control = np.full(len(Power),0) * AFQ_RICE
    RICE_Motor = np.full(len(Power),2.5) * AFQ_RICE
    RICE_Insulation = np.full(len(Power),0) * AFQ_RICE
    RICE_Catalytic = np.full(len(Power),5) * AFQ_RICE
    RICE_Electric = np.full(len(Power),1) * AFQ_RICE
    RICE_Generator = np.full(len(Power),1) * AFQ_RICE
    RICE_Sanitary = np.full(len(Power),1) * AFQ_RICE
    RICE_Storage = np.full(len(Power),2) * AFQ_RICE
elif Power.all(0) >= 25 and Power.all(0) < 100:
    mass = 5133.22
    RICE_Air = np.full(len(Power),0.4) * AFQ_RICE
    RICE_Control = np.full(len(Power),0.4) * AFQ_RICE
    RICE_Motor = np.full(len(Power),0.3) * AFQ_RICE
    RICE_Insulation = np.full(len(Power),0.4) * AFQ_RICE
    RICE_Catalytic = np.full(len(Power),0) * AFQ_RICE
    RICE_Electric = np.full(len(Power),0.4) * AFQ_RICE
    RICE_Generator = np.full(len(Power),0.3) * AFQ_RICE
    RICE_Sanitary = np.full(len(Power),0.4) * AFQ_RICE
    RICE_Storage = np.full(len(Power),0.8) * AFQ_RICE
elif Power.all(0) >= 100 and Power.all(0) < 300:
    mass = 14070.56
    RICE_Air = np.full(len(Power),1) * AFQ_RICE
    RICE_Control = np.full(len(Power),1) * AFQ_RICE
    RICE_Motor = np.full(len(Power),1.3) * AFQ_RICE
    RICE_Insulation = np.full(len(Power),1) * AFQ_RICE
    RICE_Catalytic = np.full(len(Power),0) * AFQ_RICE
    RICE_Electric = np.full(len(Power),1) * AFQ_RICE
    RICE_Generator = np.full(len(Power),1.3) * AFQ_RICE
    RICE_Sanitary = np.full(len(Power),1) * AFQ_RICE
    RICE_Storage = np.full(len(Power),2) * AFQ_RICE
elif Power.all(0) >= 300 and Power.all(0) < 700:
    mass = 25247.44
    RICE_Air = np.full(len(Power),1.7) * AFQ_RICE
    RICE_Control = np.full(len(Power),1.7) * AFQ_RICE
    RICE_Motor = np.full(len(Power), 2.8) * AFQ_RICE
    RICE_Insulation = np.full(len(Power),1.7) * AFQ_RICE
    RICE_Catalytic = np.full(len(Power),0) * AFQ_RICE
    RICE_Electric = np.full(len(Power),1.7) * AFQ_RICE
    RICE_Generator = np.full(len(Power),2.8) * AFQ_RICE
    RICE_Sanitary = np.full(len(Power),1.7) * AFQ_RICE
    RICE_Storage = np.full(len(Power),3.4) * AFQ_RICE
elif Power.all(0) >= 700 and Power.all(0) < 1400:
    mass = 40903.44
    RICE_Air = np.full(len(Power),2.6) * AFQ_RICE
    RICE_Control = np.full(len(Power),2.6) * AFQ_RICE
    RICE_Motor = np.full(len(Power),5.3) * AFQ_RICE
    RICE_Insulation = np.full(len(Power),2.6) * AFQ_RICE
    RICE_Catalytic = np.full(len(Power),0) * AFQ_RICE
    RICE_Electric = np.full(len(Power),2.6) * AFQ_RICE
    RICE_Generator = np.full(len(Power),5.3) * AFQ_RICE
    RICE_Sanitary = np.full(len(Power),2.6) * AFQ_RICE
    RICE_Storage = np.full(len(Power),5.2) * AFQ_RICE
else:
    mass = 40903.44
    i = np.ceil(Power/1400)
    RICE_Air = np.full(len(Power),2.6) * i * AFQ_RICE
    RICE_Control = np.full(len(Power),2.6) * i * AFQ_RICE
    RICE_Motor = np.full(len(Power),5.3) * i * AFQ_RICE
    RICE_Insulation = np.full(len(Power),2.6) * i * AFQ_RICE
    RICE_Catalytic = np.full(len(Power),0) * i * AFQ_RICE
    RICE_Electric = np.full(len(Power),2.6) * i * AFQ_RICE
    RICE_Generator = np.full(len(Power),5.3) * i * AFQ_RICE
    RICE_Sanitary = np.full(len(Power),2.6) * i * AFQ_RICE
    RICE_Storage = np.full(len(Power),5.2) * i * AFQ_RICE

## 3.2 Module A4

In [8]:
mass = np.full(len(Power), mass)
RICE_Truk_16_32 = ((500 + 200)*(mass)/1000) * AFQ_RICE
RICE_Light_Truk = (50*(mass)/1000) * AFQ_RICE

## 3.3 Module A5

In [9]:
if Power.all(0) < 25:
    RICE_energy = np.full(len(Power),0) * AFQ_RICE
    RICE_construction = np.full(len(Power),0) * AFQ_RICE
    RICE_planning = np.full(len(Power),1) * AFQ_RICE
    RICE_startup = np.full(len(Power),1) * AFQ_RICE
elif Power.all(0) >= 25 and Power.all(0) < 100:
    RICE_energy = np.full(len(Power),0.414) * AFQ_RICE
    RICE_construction = np.full(len(Power),0.4) * AFQ_RICE
    RICE_planning = np.full(len(Power),0.414) * AFQ_RICE
    RICE_startup = np.full(len(Power),0.414) * AFQ_RICE
elif Power.all(0) >= 100 and Power.all(0) < 300:
    RICE_energy = np.full(len(Power),1) * AFQ_RICE
    RICE_construction = np.full(len(Power),1) * AFQ_RICE
    RICE_planning = np.full(len(Power),1) * AFQ_RICE
    RICE_startup = np.full(len(Power),1) * AFQ_RICE
elif Power.all(0) >= 300 and Power.all(0) < 700:
    RICE_energy = np.full(len(Power),1) * AFQ_RICE
    RICE_construction = np.full(len(Power),1) * AFQ_RICE
    RICE_planning = np.full(len(Power),1) * AFQ_RICE
    RICE_startup = np.full(len(Power),1) * AFQ_RICE
elif Power.all(0) >= 700 and Power.all(0) < 1400:
    RICE_energy = np.full(len(Power),1.7) * AFQ_RICE
    RICE_construction = np.full(len(Power),1.7) * AFQ_RICE
    RICE_planning = np.full(len(Power),1.7) * AFQ_RICE
    RICE_startup = np.full(len(Power),1.7) * AFQ_RICE
else:
    RICE_energy = np.full(len(Power),2.61 * i) * AFQ_RICE
    RICE_construction = np.full(len(Power),2.61 * i) * AFQ_RICE
    RICE_planning = np.full(len(Power),2.61 * i) * AFQ_RICE
    RICE_startup = np.full(len(Power),2.61 * i) * AFQ_RICE

## 3.4 Module B2

In [10]:
if Power.all(0) < 25:
    RICE_maintenance = np.full(len(Power),1) * AFQ_RICE
    RICE_lubricatingoil = ((0.03/1000) * EnergyProducedHHV  *3.6) * AFQ_RICE
elif Power.all(0) >= 25 and Power.all(0) < 100:
    RICE_maintenance = np.full(len(Power),0.377) * AFQ_RICE
    RICE_lubricatingoil = ((0.03/1000) * EnergyProducedHHV  *3.6) * AFQ_RICE
elif Power.all(0) >= 100 and Power.all(0) < 300:
    RICE_maintenance = np.full(len(Power),1) * AFQ_RICE
    RICE_lubricatingoil = ((0.03/1000) * EnergyProducedHHV  *3.6) * AFQ_RICE
elif Power.all(0) >= 300 and Power.all(0) < 700:
    RICE_maintenance = np.full(len(Power),1.8) * AFQ_RICE
    RICE_lubricatingoil = ((0.03/1000) * EnergyProducedHHV  *3.6) * AFQ_RICE
elif Power.all(0) >= 700 and Power.all(0) < 1400:
    RICE_maintenance = np.full(len(Power),2.89) * AFQ_RICE
    RICE_lubricatingoil = ((0.03/1000) * EnergyProducedHHV  *3.6) * AFQ_RICE
else:
    RICE_maintenance = np.full(len(Power), 2.89 * i) * AFQ_RICE
    RICE_lubricatingoil = ((0.03/1000) * EnergyProducedHHV  *3.6) * AFQ_RICE

## 3.5 Module B4

In [11]:
RICENumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFRICE = (RSP / ReqSL)  # Scaling factor

## 3.6 Module B6

In [12]:
#Parameters for repartition of the energy input
GasBlend = EnergyInputSourceGasBlend / (EnergyInputSourceGasBlend + EnergyInputSourceBiogas + EnergyInputSourceHydrogen)
Biogas = EnergyInputSourceBiogas / (EnergyInputSourceGasBlend + EnergyInputSourceBiogas + EnergyInputSourceHydrogen)
Hydrogen = EnergyInputSourceHydrogen / (EnergyInputSourceGasBlend + EnergyInputSourceBiogas + EnergyInputSourceHydrogen)


In [13]:
x = 1
TotEnergyNeeds = np.array([EnergyProducedHHV])
TotEnergyNeeds = TotEnergyNeeds.flatten()
y = (TotEnergyNeeds * 3.6).flatten() * AFQ_RICE

RICE_EnergyvectorNG = (((1 / (HHV)) * 0.95) * A) * x * y * GasBlend
RICE_EnergyvectorBiomethane = (((1 / (HHV)) * 0.042) * B) * x * y * GasBlend
RICE_EnergyvectorHydrogen = ((((1 / 0.9) * (1 / 3.6)) * 120) * x * y) * C * GasBlend #120 MJ/kg
RICE_EnergyvectorInputBiogas = ((1 / (HHV))) * x * y * Biogas
RICE_EnergyVectorInputHydrogen = ((((1 / 0.9) * (1 / 3.6)) * 120) * x * y) * Hydrogen #120 MJ/kg
RICE_Acetaldehyde = (0.0000000008 * x * y).flatten()
RICE_AceticAcid = (0.000000121 * x * y).flatten()
RICE_Benzene = (0.000000000926 * x * y).flatten()
RICE_Benzoapyrene = (0.000000000000529 * x * y).flatten()
RICE_Butane = (0.000000926 * x * y).flatten()
RICE_CarbonDioxideFossil = (0.0561 * x * y).flatten()
RICE_CarbonMonoxideFossil = (0.000003 * x * y).flatten()
RICE_DinitrogenMonoxide = (0.000001 * x * y).flatten()
RICE_Ethane = (0.00000137 * x * y).flatten()
RICE_Formaldehyde = (0.0000000331 * x * y).flatten()
RICE_Hexane = (0.000000793 * x * y).flatten()
RICE_Mercury = (0.00000000003 * x * y).flatten()
RICE_MethaneFossil = (0.000001 * x * y).flatten()
RICE_NitrogenOxides = (0.000064 * x * y).flatten()
RICE_PAH = (0.000000008 * x * y).flatten()
RICE_Particulates = (0.0000005 * x * y).flatten()
RICE_Pentane = (0.00000115 * x * y).flatten()
RICE_Propane = (0.000000705 * x * y).flatten()
RICE_PropionicAcid = (0.000000016 * x * y).flatten()
RICE_SulfurDioxide = (0.0000005 * x * y).flatten()
RICE_Toluene = (0.0000000015 * x * y).flatten()
RICE_Dioxins = (0.000000000029 * x * y).flatten()

## 3.7 Module C2

In [14]:
RICE_Transport_to_treatment_plant = 50 * (mass/1000)

## 3.8 Module C3

In [15]:
if Power.all(0) < 25:
    RICE_reinforcingsteelAirinput = np.full(len(Power), 288 * 0) * AFQ_RICE
    RICE_aluminiumControlcabinet = np.full(len(Power), 0.15 * 0) * AFQ_RICE
    RICE_copperControlcabinet = np.full(len(Power), 10.8 * 0) * AFQ_RICE
    RICE_leadControlcabinet = np.full(len(Power), 0.76 * 0) * AFQ_RICE
    RICE_nickelControlcabinet = np.full(len(Power), 0.34 * 0) * AFQ_RICE
    RICE_platinumControlcabinet = np.full(len(Power), 0.0021 * 0) * AFQ_RICE
    RICE_HDPEControlcabinet = np.full(len(Power), 78.5 * 0) * AFQ_RICE
    RICE_PVCemulsionatedControlcabinet = np.full(len(Power), 0.95745 * 0) * AFQ_RICE
    RICE_PVCpolimerizedControlcabinet = np.full(len(Power), 6.54255 * 0) * AFQ_RICE
    RICE_reinforcingsteelControlcabinet = np.full(len(Power), 276 * 0) * AFQ_RICE
    RICE_tinControlcabinet = np.full(len(Power), 1.62 * 0) * AFQ_RICE
    RICE_zincControlcabinet = np.full(len(Power), 0.25 * 0) * AFQ_RICE
    RICE_castironGasmotor = np.full(len(Power), 1000 * 2.5) * AFQ_RICE
    RICE_reinforcingsteelGasmotor = np.full(len(Power), 100 * 2.5) * AFQ_RICE
    RICE_chromiumsteelGasmotor = np.full(len(Power), 100 * 2.5) * AFQ_RICE
    RICE_lowalloyedsteelGasmotor = np.full(len(Power), 200 * 2.5) * AFQ_RICE
    RICE_reinforcingsteelSoundinsulation = np.full(len(Power), 1920 * 0) * AFQ_RICE
    RICE_stonewoolSoundinsulation = np.full(len(Power), 480 * 0) * AFQ_RICE
    RICE_corrugatedboxboardConverter = np.full(len(Power), 0.55 * 5) * AFQ_RICE
    RICE_palladiumConverter = np.full(len(Power), 0.0082 * 5) * AFQ_RICE
    RICE_platinumConverter = np.full(len(Power), 0.0409 * 5) * AFQ_RICE
    RICE_rhodiumConverter = np.full(len(Power), 0.0082 * 5) * AFQ_RICE
    RICE_chromiumsteelConverter = np.full(len(Power), 66 * 5) * AFQ_RICE
    RICE_zeoliteConverter = np.full(len(Power), 3.8 * 5) * AFQ_RICE
    RICE_aluminiumEle = np.full(len(Power), 0.1 * 1) * AFQ_RICE
    RICE_copperEle = np.full(len(Power), 6.5 * 1) * AFQ_RICE
    RICE_leadEle = np.full(len(Power), 0.47 * 1) * AFQ_RICE
    RICE_nickelEle = np.full(len(Power), 0.21 * 1) * AFQ_RICE
    RICE_platinumEle = np.full(len(Power), 0.0013 * 1) * AFQ_RICE
    RICE_HDPEEle = np.full(len(Power), 47.4 * 1) * AFQ_RICE
    RICE_PVCpolimerizedEle = np.full(len(Power), 0.501133 * 1) * AFQ_RICE
    RICE_reinforcingsteelEle = np.full(len(Power), 52.7 * 1) * AFQ_RICE
    RICE_tinEle = np.full(len(Power), 0.997 * 1) * AFQ_RICE
    RICE_zincEle = np.full(len(Power), 0.154 * 1) * AFQ_RICE
    RICE_castironGen = np.full(len(Power), 595 * 1) * AFQ_RICE
    RICE_copperGen = np.full(len(Power), 255 * 1) * AFQ_RICE
    RICE_reinforcingsteelSan = np.full(len(Power), 353 * 1) * AFQ_RICE
    RICE_stonewoolSan = np.full(len(Power), 7 * 1) * AFQ_RICE
    RICE_alkydpaintStorage = np.full(len(Power), 4.2 * 2) * AFQ_RICE
    RICE_aluminiumStorage = np.full(len(Power), 87.2 * 2) * AFQ_RICE
    RICE_reinforcingsteelStorage = np.full(len(Power), 1770 * 2) * AFQ_RICE
    RICE_sheetrollingaluminiumStorage = np.full(len(Power), 87.2 * 2) * AFQ_RICE
    RICE_sheetrollingsteelStorage = np.full(len(Power), 1770 * 2) * AFQ_RICE
    RICE_chromiumsteelStorage = np.full(len(Power), 3.7 * 2) * AFQ_RICE
    RICE_stonewoolStorage = np.full(len(Power), 84 * 2) * AFQ_RICE
elif Power.all(0) >= 25 and Power.all(0) < 100:
    RICE_reinforcingsteelAirinput = np.full(len(Power), 288 * 0.4) * AFQ_RICE
    RICE_aluminiumControlcabinet = np.full(len(Power), 0.15 * 0.4) * AFQ_RICE
    RICE_copperControlcabinet = np.full(len(Power), 10.8 * 0.4) * AFQ_RICE
    RICE_leadControlcabinet = np.full(len(Power), 0.76 * 0.4) * AFQ_RICE
    RICE_nickelControlcabinet = np.full(len(Power), 0.34 * 0.4) * AFQ_RICE
    RICE_platinumControlcabinet = np.full(len(Power), 0.0021 * 0.4) * AFQ_RICE
    RICE_HDPEControlcabinet = np.full(len(Power), 78.5 * 0.4) * AFQ_RICE
    RICE_PVCemulsionatedControlcabinet = np.full(len(Power), 0.95745 * 0.4) * AFQ_RICE
    RICE_PVCpolimerizedControlcabinet = np.full(len(Power), 6.54255 * 0.4) * AFQ_RICE
    RICE_reinforcingsteelControlcabinet = np.full(len(Power), 276 * 0.4) * AFQ_RICE
    RICE_tinControlcabinet = np.full(len(Power), 1.62 * 0.4) * AFQ_RICE
    RICE_zincControlcabinet = np.full(len(Power), 0.25 * 0.4) * AFQ_RICE
    RICE_castironGasmotor = np.full(len(Power), 1000 * 0.3) * AFQ_RICE
    RICE_reinforcingsteelGasmotor = np.full(len(Power), 100 * 0.3) * AFQ_RICE
    RICE_chromiumsteelGasmotor = np.full(len(Power), 100 * 0.3) * AFQ_RICE
    RICE_lowalloyedsteelGasmotor = np.full(len(Power), 200 * 0.3) * AFQ_RICE
    RICE_reinforcingsteelSoundinsulation = np.full(len(Power), 1920 * 0.4) * AFQ_RICE
    RICE_stonewoolSoundinsulation = np.full(len(Power), 480 * 0.4) * AFQ_RICE
    RICE_corrugatedboxboardConverter = np.full(len(Power), 0.55 * 0) * AFQ_RICE
    RICE_palladiumConverter = np.full(len(Power), 0.0082 * 0) * AFQ_RICE
    RICE_platinumConverter = np.full(len(Power), 0.0409 * 0) * AFQ_RICE
    RICE_rhodiumConverter = np.full(len(Power), 0.0082 * 0) * AFQ_RICE
    RICE_chromiumsteelConverter = np.full(len(Power), 66 * 0) * AFQ_RICE
    RICE_zeoliteConverter = np.full(len(Power), 3.8 * 0) * AFQ_RICE
    RICE_aluminiumEle = np.full(len(Power), 0.1 * 0.4) * AFQ_RICE
    RICE_copperEle = np.full(len(Power), 6.5 * 0.4) * AFQ_RICE
    RICE_leadEle = np.full(len(Power), 0.47 * 0.4) * AFQ_RICE
    RICE_nickelEle = np.full(len(Power), 0.21 * 0.4) * AFQ_RICE
    RICE_platinumEle = np.full(len(Power), 0.0013 * 0.4) * AFQ_RICE
    RICE_HDPEEle = np.full(len(Power), 47.4 * 0.4) * AFQ_RICE
    RICE_PVCpolimerizedEle = np.full(len(Power), 0.501133 * 0.4) * AFQ_RICE
    RICE_reinforcingsteelEle = np.full(len(Power), 52.7 * 0.4) * AFQ_RICE
    RICE_tinEle = np.full(len(Power), 0.997 * 0.4) * AFQ_RICE
    RICE_zincEle = np.full(len(Power), 0.154 * 0.4) * AFQ_RICE
    RICE_castironGen = np.full(len(Power), 595 * 0.3) * AFQ_RICE
    RICE_copperGen = np.full(len(Power), 255 * 0.3) * AFQ_RICE
    RICE_reinforcingsteelSan = np.full(len(Power), 353 * 0.4) * AFQ_RICE
    RICE_stonewoolSan = np.full(len(Power), 7 * 0.4) * AFQ_RICE
    RICE_alkydpaintStorage = np.full(len(Power), 4.2 * 0.8) * AFQ_RICE
    RICE_aluminiumStorage = np.full(len(Power), 87.2 * 0.8) * AFQ_RICE
    RICE_reinforcingsteelStorage = np.full(len(Power), 1770 * 0.8) * AFQ_RICE
    RICE_sheetrollingaluminiumStorage = np.full(len(Power), 87.2 * 0.8) * AFQ_RICE
    RICE_sheetrollingsteelStorage = np.full(len(Power), 1770 * 0.8) * AFQ_RICE
    RICE_chromiumsteelStorage = np.full(len(Power), 3.7 * 0.8) * AFQ_RICE
    RICE_stonewoolStorage = np.full(len(Power), 84 * 0.8) * AFQ_RICE
elif Power.all(0) >= 100 and Power.all(0) < 300:
    RICE_reinforcingsteelAirinput = np.full(len(Power), 288 * 1) * AFQ_RICE
    RICE_aluminiumControlcabinet = np.full(len(Power), 0.15 * 1) * AFQ_RICE
    RICE_copperControlcabinet = np.full(len(Power), 10.8 * 1) * AFQ_RICE
    RICE_leadControlcabinet = np.full(len(Power), 0.76 * 1) * AFQ_RICE
    RICE_nickelControlcabinet = np.full(len(Power), 0.34 * 1) * AFQ_RICE
    RICE_platinum = np.full(len(Power), 0.0021 * 1) * AFQ_RICE
    RICE_HDPE = np.full(len(Power), 78.5 * 1) * AFQ_RICE
    RICE_PVCemulsionatedControlcabinet = np.full(len(Power), 0.95745 * 1) * AFQ_RICE
    RICE_PVCpolimerizedControlcabinet = np.full(len(Power), 6.54255 * 1) * AFQ_RICE
    RICE_reinforcingsteelControlcabinet = np.full(len(Power), 276 * 1) * AFQ_RICE
    RICE_tinControlcabinet = np.full(len(Power), 1.62 * 1) * AFQ_RICE
    RICE_zincControlcabinet = np.full(len(Power), 0.25 * 1) * AFQ_RICE
    RICE_castironGasmotor = np.full(len(Power), 1000 * 1.3) * AFQ_RICE
    RICE_reinforcingsteelGasmotor = np.full(len(Power), 100 * 1.3) * AFQ_RICE
    RICE_chromiumsteelGasmotor = np.full(len(Power), 100 * 1.3) * AFQ_RICE
    RICE_lowalloyedsteelGasmotor = np.full(len(Power), 200 * 1.3) * AFQ_RICE
    RICE_reinforcingsteelSoundinsulation = np.full(len(Power), 1920 * 1) * AFQ_RICE
    RICE_stonewool = np.full(len(Power), 480 * 1) * AFQ_RICE
    RICE_corrugatedboxboardConverter = np.full(len(Power), 0.55 * 0) * AFQ_RICE
    RICE_palladiumConverter = np.full(len(Power), 0.0082 * 0) * AFQ_RICE
    RICE_platinumConverter = np.full(len(Power), 0.0409 * 0) * AFQ_RICE
    RICE_rhodiumConverter = np.full(len(Power), 0.0082 * 0) * AFQ_RICE
    RICE_chromiumsteelConverter = np.full(len(Power), 66 * 0) * AFQ_RICE
    RICE_zeoliteConverter = np.full(len(Power), 3.8 * 0) * AFQ_RICE
    RICE_aluminiumEle = np.full(len(Power), 0.1 * 1) * AFQ_RICE
    RICE_copperEle = np.full(len(Power), 6.5 * 1) * AFQ_RICE
    RICE_leadEle = np.full(len(Power), 0.47 * 1) * AFQ_RICE
    RICE_nickelEle = np.full(len(Power), 0.21 * 1) * AFQ_RICE
    RICE_platinumEle = np.full(len(Power), 0.0013 * 1) * AFQ_RICE
    RICE_HDPEEle = np.full(len(Power), 47.4 * 1) * AFQ_RICE
    RICE_PVCpolimerizedEle = np.full(len(Power), 0.501133 * 1) * AFQ_RICE
    RICE_reinforcingsteelEle = np.full(len(Power), 52.7 * 1) * AFQ_RICE
    RICE_tinEle = np.full(len(Power), 0.997 * 1) * AFQ_RICE
    RICE_zincEle = np.full(len(Power), 0.154 * 1) * AFQ_RICE
    RICE_castironGen = np.full(len(Power), 595 * 1.3) * AFQ_RICE
    RICE_copperGen = np.full(len(Power), 255 * 1.3) * AFQ_RICE
    RICE_reinforcingsteelSan = np.full(len(Power), 353 * 1) * AFQ_RICE
    RICE_stonewoolSan = np.full(len(Power), 7 * 1) * AFQ_RICE
    RICE_alkydpaintStorage = np.full(len(Power), 4.2 * 2) * AFQ_RICE
    RICE_aluminiumStorage = np.full(len(Power), 87.2 * 2) * AFQ_RICE
    RICE_reinforcingsteelStorage = np.full(len(Power), 1770 * 2) * AFQ_RICE
    RICE_sheetrollingaluminiumStorage = np.full(len(Power), 87.2 * 2) * AFQ_RICE
    RICE_sheetrollingsteelStorage = np.full(len(Power), 1770 * 2) * AFQ_RICE
    RICE_chromiumsteelStorage = np.full(len(Power), 3.7 * 2) * AFQ_RICE
    RICE_stonewoolStorage = np.full(len(Power), 84 * 2) * AFQ_RICE
elif Power.all(0) >= 300 and Power.all(0) < 700:
    RICE_reinforcingsteelAirinput = np.full(len(Power), 288 * 1.7) * AFQ_RICE
    RICE_aluminiumControlcabinet = np.full(len(Power), 0.15 * 1.7) * AFQ_RICE
    RICE_copperControlcabinet = np.full(len(Power), 10.8 * 1.7) * AFQ_RICE
    RICE_leadControlcabinet = np.full(len(Power), 0.76 * 1.7) * AFQ_RICE
    RICE_nickelControlcabinet = np.full(len(Power), 0.34 * 1.7) * AFQ_RICE
    RICE_platinumControlcabinet = np.full(len(Power), 0.0021 * 1.7) * AFQ_RICE
    RICE_HDPEControlcabinet = np.full(len(Power), 78.5 * 1.7) * AFQ_RICE
    RICE_PVCemulsionatedControlcabinet = np.full(len(Power), 0.95745 * 1.7) * AFQ_RICE
    RICE_PVCpolimerizedControlcabinet = np.full(len(Power), 6.54255 * 1.7) * AFQ_RICE
    RICE_reinforcingsteelControlcabinet = np.full(len(Power), 276 * 1.7) * AFQ_RICE
    RICE_tinControlcabinet = np.full(len(Power), 1.62 * 1.7) * AFQ_RICE
    RICE_zincControlcabinet = np.full(len(Power), 0.25 * 1.7) * AFQ_RICE
    RICE_castironGasmotor = np.full(len(Power), 1000 * 2.8) * AFQ_RICE
    RICE_reinforcingsteelGasmotor = np.full(len(Power), 100 * 2.8) * AFQ_RICE
    RICE_chromiumsteelGasmotor = np.full(len(Power), 100 * 2.8) * AFQ_RICE
    RICE_lowalloyedsteelGasmotor = np.full(len(Power), 200 * 2.8) * AFQ_RICE
    RICE_reinforcingsteelSoundinsulation = np.full(len(Power), 1920 * 1.7) * AFQ_RICE
    RICE_stonewoolSoundinsulation = np.full(len(Power), 480 * 1.7) * AFQ_RICE
    RICE_corrugatedboxboardConverter = np.full(len(Power), 0.55 * 0) * AFQ_RICE
    RICE_palladiumConverter = np.full(len(Power), 0.0082 * 0) * AFQ_RICE
    RICE_platinumConverter = np.full(len(Power), 0.0409 * 0) * AFQ_RICE
    RICE_rhodiumConverter = np.full(len(Power), 0.0082 * 0) * AFQ_RICE
    RICE_chromiumsteelConverter = np.full(len(Power), 66 * 0) * AFQ_RICE
    RICE_zeoliteConverter = np.full(len(Power), 3.8 * 0) * AFQ_RICE
    RICE_aluminiumEle = np.full(len(Power), 0.1 * 1.7) * AFQ_RICE
    RICE_copperEle = np.full(len(Power), 6.5 * 1.7) * AFQ_RICE
    RICE_leadEle = np.full(len(Power), 0.47 * 1.7) * AFQ_RICE
    RICE_nickelEle = np.full(len(Power), 0.21 * 1.7) * AFQ_RICE
    RICE_platinumEle = np.full(len(Power), 0.0013 * 1.7) * AFQ_RICE
    RICE_HDPEEle = np.full(len(Power), 47.4 * 1.7) * AFQ_RICE
    RICE_PVCpolimerizedEle = np.full(len(Power), 0.501133 * 1.7) * AFQ_RICE
    RICE_reinforcingsteelEle = np.full(len(Power), 52.7 * 1.7) * AFQ_RICE
    RICE_tinEle = np.full(len(Power), 0.997 * 1.7) * AFQ_RICE
    RICE_zincEle = np.full(len(Power), 0.154 * 1.7) * AFQ_RICE
    RICE_castironGen = np.full(len(Power), 595 * 2.8) * AFQ_RICE
    RICE_copperGen = np.full(len(Power), 255 * 2.8) * AFQ_RICE
    RICE_reinforcingsteelSan = np.full(len(Power), 353 * 1.7) * AFQ_RICE
    RICE_stonewoolSan = np.full(len(Power), 7 * 1.7) * AFQ_RICE
    RICE_alkydpaintStorage = np.full(len(Power), 4.2 * 3.4) * AFQ_RICE
    RICE_aluminiumStorage = np.full(len(Power), 87.2 * 3.4) * AFQ_RICE
    RICE_reinforcingsteelStorage = np.full(len(Power), 1770 * 3.4) * AFQ_RICE
    RICE_sheetrollingaluminiumStorage = np.full(len(Power), 87.2 * 3.4) * AFQ_RICE
    RICE_sheetrollingsteelStorage = np.full(len(Power), 1770 * 3.4) * AFQ_RICE
    RICE_chromiumsteelStorage = np.full(len(Power), 3.7 * 3.4) * AFQ_RICE
    RICE_stonewoolStorage = np.full(len(Power), 84 * 3.4) * AFQ_RICE

elif Power.all(0) >= 700 and Power.all(0) < 1400:
    RICE_reinforcingsteelAirinput = np.full(len(Power), 288 * 2.6) * AFQ_RICE
    RICE_aluminiumControlcabinet = np.full(len(Power), 0.15 * 2.6) * AFQ_RICE
    RICE_copperControlcabinet = np.full(len(Power), 10.8 * 2.6) * AFQ_RICE
    RICE_leadControlcabinet = np.full(len(Power), 0.76 * 2.6) * AFQ_RICE
    RICE_nickelControlcabinet = np.full(len(Power), 0.34 * 2.6) * AFQ_RICE
    RICE_platinumControlcabinet = np.full(len(Power), 0.0021 * 2.6) * AFQ_RICE
    RICE_HDPEControlcabinet = np.full(len(Power), 78.5 * 2.6) * AFQ_RICE
    RICE_PVCemulsionatedControlcabinet = np.full(len(Power), 0.95745 * 2.6) * AFQ_RICE
    RICE_PVCpolimerizedControlcabinet = np.full(len(Power), 6.54255 * 2.6) * AFQ_RICE
    RICE_reinforcingsteelControlcabinet = np.full(len(Power), 276 * 2.6) * AFQ_RICE
    RICE_tinControlcabinet = np.full(len(Power), 1.62 * 2.6) * AFQ_RICE
    RICE_zincControlcabinet = np.full(len(Power), 0.25 * 2.6) * AFQ_RICE
    RICE_castironGasmotor = np.full(len(Power), 1000 * 5.3) * AFQ_RICE
    RICE_reinforcingsteelGasmotor = np.full(len(Power), 100 * 5.3) * AFQ_RICE
    RICE_chromiumsteelGasmotor = np.full(len(Power), 100 * 5.3) * AFQ_RICE
    RICE_lowalloyedsteelGasmotor = np.full(len(Power), 200 * 5.3) * AFQ_RICE
    RICE_reinforcingsteelSoundinsulation = np.full(len(Power), 1920 * 2.6) * AFQ_RICE
    RICE_stonewoolSoundinsulation = np.full(len(Power), 480 * 2.6) * AFQ_RICE
    RICE_corrugatedboxboardConverter = np.full(len(Power), 0.55 * 0) * AFQ_RICE
    RICE_palladiumConverter = np.full(len(Power), 0.0082 * 0) * AFQ_RICE
    RICE_platinumConverter = np.full(len(Power), 0.0409 * 0) * AFQ_RICE
    RICE_rhodiumConverter = np.full(len(Power), 0.0082 * 0) * AFQ_RICE
    RICE_chromiumsteelConverter = np.full(len(Power), 66 * 0) * AFQ_RICE
    RICE_zeoliteConverter = np.full(len(Power), 3.8 * 0) * AFQ_RICE
    RICE_aluminiumEle = np.full(len(Power), 0.1 * 2.6) * AFQ_RICE
    RICE_copperEle = np.full(len(Power), 6.5 * 2.6) * AFQ_RICE
    RICE_leadEle = np.full(len(Power), 0.47 * 2.6) * AFQ_RICE
    RICE_nickelEle = np.full(len(Power), 0.21 * 2.6) * AFQ_RICE
    RICE_platinumEle = np.full(len(Power), 0.0013 * 2.6) * AFQ_RICE
    RICE_HDPEEle = np.full(len(Power), 47.4 * 2.6) * AFQ_RICE
    RICE_PVCpolimerizedEle = np.full(len(Power), 0.501133 * 2.6) * AFQ_RICE
    RICE_reinforcingsteelEle = np.full(len(Power), 52.7 * 2.6) * AFQ_RICE
    RICE_tinEle = np.full(len(Power), 0.997 * 2.6) * AFQ_RICE
    RICE_zincEle = np.full(len(Power), 0.154 * 2.6) * AFQ_RICE
    RICE_castironGen = np.full(len(Power), 595 * 5.3) * AFQ_RICE
    RICE_copperGen = np.full(len(Power), 255 * 5.3) * AFQ_RICE
    RICE_reinforcingsteelSan = np.full(len(Power), 353 * 2.6) * AFQ_RICE
    RICE_stonewoolSan = np.full(len(Power), 7 * 2.6) * AFQ_RICE
    RICE_alkydpaintStorage = np.full(len(Power), 4.2 * 5.2) * AFQ_RICE
    RICE_aluminiumStorage = np.full(len(Power), 87.2 * 5.2) * AFQ_RICE
    RICE_reinforcingsteelStorage = np.full(len(Power), 1770 * 5.2) * AFQ_RICE
    RICE_sheetrollingaluminiumStorage = np.full(len(Power), 87.2 * 5.2) * AFQ_RICE
    RICE_sheetrollingsteelStorage = np.full(len(Power), 1770 * 5.2) * AFQ_RICE
    RICE_chromiumsteelStorage = np.full(len(Power), 3.7 * 5.2) * AFQ_RICE
    RICE_stonewoolStorage = np.full(len(Power), 84 * 5.2) * AFQ_RICE
else:
    RICE_reinforcingsteelAirinput = np.full(len(Power), 288 * 2.6 * i) * AFQ_RICE
    RICE_aluminiumControlcabinet = np.full(len(Power), 0.15 * 2.6 * i) * AFQ_RICE
    RICE_copperControlcabinet = np.full(len(Power), 10.8 * 2.6 * i) * AFQ_RICE
    RICE_leadControlcabinet = np.full(len(Power), 0.76 * 2.6 * i) * AFQ_RICE
    RICE_nickelControlcabinet = np.full(len(Power), 0.34 * 2.6 * i) * AFQ_RICE
    RICE_platinumControlcabinet = np.full(len(Power), 0.0021 * 2.6 * i) * AFQ_RICE
    RICE_HDPEControlcabinet = np.full(len(Power), 78.5 * 2.6 * i) * AFQ_RICE
    RICE_PVCemulsionatedControlcabinet = np.full(len(Power), 0.95745 * 2.6 * i) * AFQ_RICE
    RICE_PVCpolimerizedControlcabinet = np.full(len(Power), 6.54255 * 2.6 * i) * AFQ_RICE
    RICE_reinforcingsteelControlcabinet = np.full(len(Power), 276 * 2.6 * i) * AFQ_RICE
    RICE_tinControlcabinet = np.full(len(Power), 1.62 * 2.6 * i) * AFQ_RICE
    RICE_zincControlcabinet = np.full(len(Power), 0.25 * 2.6 * i) * AFQ_RICE
    RICE_castironGasmotor = np.full(len(Power), 1000 * 5.3 * i) * AFQ_RICE
    RICE_reinforcingsteelGasmotor = np.full(len(Power), 100 * 5.3 * i) * AFQ_RICE
    RICE_chromiumsteelGasmotor = np.full(len(Power), 100 * 5.3 * i) * AFQ_RICE
    RICE_lowalloyedsteelGasmotor = np.full(len(Power), 200 * 5.3 * i) * AFQ_RICE
    RICE_reinforcingsteelSoundinsulation = np.full(len(Power), 1920 * 2.6 * i) * AFQ_RICE
    RICE_stonewoolSoundinsulation = np.full(len(Power), 480 * 2.6 * i) * AFQ_RICE
    RICE_corrugatedboxboardConverter = np.full(len(Power), 0.55 * 0 * i) * AFQ_RICE
    RICE_palladiumConverter = np.full(len(Power), 0.0082 * 0 * i) * AFQ_RICE
    RICE_platinumConverter = np.full(len(Power), 0.0409 * 0 * i) * AFQ_RICE
    RICE_rhodiumConverter = np.full(len(Power), 0.0082 * 0 * i) * AFQ_RICE
    RICE_chromiumsteelConverter = np.full(len(Power), 66 * 0 * i) * AFQ_RICE
    RICE_zeoliteConverter = np.full(len(Power), 3.8 * 0 * i) * AFQ_RICE

## 3.9 Module C4

In [16]:
RICE_EOL_landfill_reinforcingsteelAirinput = RICE_reinforcingsteelAirinput * LandfillMetals
RICE_EOL_landfill_aluminiumControlcabinet = RICE_aluminiumControlcabinet * LandfillMetals
RICE_EOL_landfill_copperControlcabinet = RICE_copperControlcabinet * LandfillMetals
RICE_EOL_landfill_leadControlcabinet = RICE_leadControlcabinet * LandfillMetals2
RICE_EOL_landfill_nickelControlcabinet = RICE_nickelControlcabinet * LandfillMetals2
RICE_EOL_landfill_platinumControlcabinet = RICE_platinumControlcabinet * LandfillMetals2
RICE_EOL_inc_HDPEControlcabinet = RICE_HDPEControlcabinet * IncinerationPEHD
RICE_EOL_landfill_HDPEControlcabinet = RICE_HDPEControlcabinet * LandfillPEHD
RICE_EOL_inc_PVCemulsionatedControlcabinet = RICE_PVCemulsionatedControlcabinet * IncinerationPVC
RICE_EOL_landfill_PVCemulsionatedControlcabinet = RICE_PVCemulsionatedControlcabinet * LandfillPVC
RICE_EOL_inc_PVCpolimerizedControlcabinet = RICE_PVCpolimerizedControlcabinet * IncinerationPVC
RICE_EOL_landfill_PVCpolimerizedControlcabinet = RICE_PVCpolimerizedControlcabinet * LandfillPVC
RICE_EOL_landfill_reinforcingsteelControlcabinet = RICE_reinforcingsteelControlcabinet * LandfillMetals
RICE_EOL_landfill_tinControlcabinet = RICE_tinControlcabinet * LandfillMetals
RICE_EOL_landfill_zincControlcabinet = RICE_zincControlcabinet * LandfillMetals2
RICE_EOL_landfill_castironGasmotor = RICE_castironGasmotor * LandfillMetals2
RICE_EOL_landfill_reinforcingsteelGasmotor = RICE_reinforcingsteelGasmotor * LandfillMetals
RICE_EOL_landfill_chromiumsteelGasmotor = RICE_chromiumsteelGasmotor * LandfillMetals2
RICE_EOL_landfill_lowalloyedsteelGasmotor = RICE_lowalloyedsteelGasmotor * LandfillMetals
RICE_EOL_landfill_reinforcingsteelSoundinsulation = RICE_reinforcingsteelSoundinsulation * LandfillMetals
RICE_EOL_landfill_stonewoolSoundinsulation = RICE_stonewoolSoundinsulation * LandfillMineralWool
RICE_EOL_inc_corrugatedboxboardConverter = RICE_corrugatedboxboardConverter * IncinerationCardboard
RICE_EOL_landfill_corrugatedboxboardConverter = RICE_corrugatedboxboardConverter * LandfillCardboard
RICE_EOL_landfill_palladiumConverter = RICE_palladiumConverter * LandfillMetals2
RICE_EOL_landfill_platinumConverter = RICE_platinumConverter * LandfillMetals2
RICE_EOL_landfill_rhodiumConverter = RICE_rhodiumConverter * LandfillMetals2
RICE_EOL_landfill_chromiumsteelConverter = RICE_chromiumsteelConverter * LandfillMetals2
RICE_EOL_inc_zeoliteConverter = RICE_zeoliteConverter
RICE_EOL_landfill_aluminiumEle = RICE_aluminiumEle * LandfillMetals
RICE_EOL_landfill_copperEle = RICE_copperEle * LandfillMetals
RICE_EOL_landfill_leadEle = RICE_leadEle * LandfillMetals2
RICE_EOL_landfill_nickelEle = RICE_nickelEle * LandfillMetals2
RICE_EOL_landfill_platinumEle = RICE_platinumEle * LandfillMetals2
RICE_EOL_inc_HDPEEle = RICE_HDPEEle * IncinerationPEHD
RICE_EOL_landfill_HDPEEle = RICE_HDPEEle * LandfillPEHD
RICE_EOL_inc_PVCpolimerizedEle = RICE_PVCpolimerizedEle * IncinerationPVC
RICE_EOL_landfill_PVCpolimerizedEle = RICE_PVCpolimerizedEle * LandfillPVC
RICE_EOL_landfill_reinforcingsteelEle = RICE_reinforcingsteelEle * LandfillMetals
RICE_EOL_landfill_tinEle = RICE_tinEle * LandfillMetals
RICE_EOL_landfill_zincEle = RICE_zincEle * LandfillMetals2
RICE_EOL_landfill_castironGen = RICE_castironGen * LandfillMetals2
RICE_EOL_landfill_copperGen = RICE_copperGen * LandfillMetals
RICE_EOL_landfill_reinforcingsteelSan = RICE_reinforcingsteelSan * LandfillMetals
RICE_EOL_landfill_stonewoolSan = RICE_stonewoolSan * LandfillMineralWool
RICE_EOL_inc_alkydpaintStorage = RICE_alkydpaintStorage
RICE_EOL_landfill_aluminiumStorage = RICE_aluminiumStorage * LandfillMetals
RICE_EOL_landfill_reinforcingsteelStorage = RICE_reinforcingsteelStorage * LandfillMetals
RICE_EOL_landfill_sheetrollingaluminiumStorage = RICE_sheetrollingaluminiumStorage * LandfillMetals
RICE_EOL_landfill_sheetrollingsteelStorage = RICE_sheetrollingsteelStorage * LandfillMetals
RICE_EOL_landfill_chromiumsteelStorage = RICE_chromiumsteelStorage * LandfillMetals2
RICE_EOL_landfill_stonewoolStorage = RICE_stonewoolStorage * LandfillMineralWool

## 3.10 Module D1 (related to the export of secondary materials)

In [17]:
RICE_recycling_reinforcingsteelAirinput = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelAirinput) - (RecycledContentMetals * RICE_reinforcingsteelAirinput)
RICE_primary_reinforcingsteelAirinput = ((RecycledContentMetals * RICE_reinforcingsteelAirinput) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelAirinput))* SRmetals

RICE_recycling_aluminiumControlcabinet = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_aluminiumControlcabinet) - (RecycledContentMetals * RICE_aluminiumControlcabinet)
RICE_primary_aluminiumControlcabinet = ((RecycledContentMetals * RICE_aluminiumControlcabinet) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_aluminiumControlcabinet))* SRmetals

RICE_recycling_copperControlcabinet = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_copperControlcabinet) - (RecycledContentMetals * RICE_copperControlcabinet)
RICE_primary_copperControlcabinet = ((RecycledContentMetals * RICE_copperControlcabinet) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_copperControlcabinet))* SRmetals

RICE_recycling_leadControlcabinet = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_leadControlcabinet) - (RecycledContentMetals * RICE_leadControlcabinet)
RICE_primary_leadControlcabinet = ((RecycledContentMetals * RICE_leadControlcabinet) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_leadControlcabinet))* SRmetals

RICE_recycling_nickelControlcabinet = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_nickelControlcabinet) - (RecycledContentMetals * RICE_nickelControlcabinet)
RICE_primary_nickelControlcabinet = ((RecycledContentMetals * RICE_nickelControlcabinet) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_nickelControlcabinet))* SRmetals

RICE_recycling_platinumControlcabinet = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_platinumControlcabinet) - (RecycledContentMetals * RICE_platinumControlcabinet)
RICE_primary_platinumControlcabinet = ((RecycledContentMetals * RICE_platinumControlcabinet) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_platinumControlcabinet)) * SRmetals

RICE_recycling_HDPEControlcabinet = (RecyclingPEHD * RecyclingEfficiencyPlastics * RICE_tinControlcabinet) - (RecycledContentPlastic * RICE_tinControlcabinet)
RICE_primary_HDPEControlcabinet = ((RecycledContentPlastic * RICE_tinControlcabinet) - (RecyclingPEHD * RecyclingEfficiencyPlastics * RICE_tinControlcabinet)) * SRmetals

RICE_recycling_reinforcingsteelControlcabinet = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelControlcabinet) - (RecycledContentMetals * RICE_reinforcingsteelControlcabinet)
RICE_primary_reinforcingsteelControlcabinet = ((RecycledContentMetals * RICE_reinforcingsteelControlcabinet) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelControlcabinet)) * SRmetals

RICE_recycling_tinControlcabinet = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_tinControlcabinet) - (RecycledContentMetals * RICE_tinControlcabinet)
RICE_primary_tinControlcabinet = ((RecycledContentMetals * RICE_tinControlcabinet) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_tinControlcabinet))* SRmetals

RICE_recycling_zincControlcabinet = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_zincControlcabinet) - (RecycledContentMetals * RICE_zincControlcabinet)
RICE_primary_zincControlcabinet = ((RecycledContentMetals * RICE_zincControlcabinet) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_zincControlcabinet))* SRmetals

RICE_recycling_castironGasmotor = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_castironGasmotor) - (RecycledContentMetals * RICE_castironGasmotor)
RICE_primary_castironGasmotor = ((RecycledContentMetals * RICE_castironGasmotor) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_castironGasmotor))* SRmetals

RICE_recycling_reinforcingsteelGasmotor = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelGasmotor) - (RecycledContentMetals * RICE_reinforcingsteelGasmotor)
RICE_primary_reinforcingsteelGasmotor = ((RecycledContentMetals * RICE_reinforcingsteelGasmotor) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelGasmotor))* SRmetals

RICE_recycling_chromiumsteelGasmotor = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_chromiumsteelGasmotor) - (RecycledContentMetals * RICE_chromiumsteelGasmotor)
RICE_primary_chromiumsteelGasmotor = ((RecycledContentMetals * RICE_chromiumsteelGasmotor) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_chromiumsteelGasmotor))* SRmetals

RICE_recycling_lowalloyedsteelGasmotor = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_lowalloyedsteelGasmotor) - (RecycledContentMetals * RICE_lowalloyedsteelGasmotor)
RICE_primary_lowalloyedsteelGasmotor = ((RecycledContentMetals * RICE_lowalloyedsteelGasmotor) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_lowalloyedsteelGasmotor))* SRmetals

RICE_recycling_reinforcingsteelSoundinsulation = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelSoundinsulation) - (RecycledContentMetals * RICE_reinforcingsteelSoundinsulation)
RICE_primary_reinforcingsteelSoundinsulation = ((RecycledContentMetals * RICE_reinforcingsteelSoundinsulation) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelSoundinsulation))* SRmetals

RICE_recycling_corrugatedboxboardConverter = (RecyclingCardboard * RecyclingEfficiencyCardboard * RICE_corrugatedboxboardConverter) - (RecycledContentCardboard * RICE_corrugatedboxboardConverter)
RICE_primary_corrugatedboxboardConverter = ((RecycledContentCardboard * RICE_corrugatedboxboardConverter) - (RecyclingCardboard * RecyclingEfficiencyCardboard * RICE_corrugatedboxboardConverter))* SRinert

RICE_recycling_palladiumConverter = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_palladiumConverter) - (RecycledContentMetals * RICE_palladiumConverter)
RICE_primary_palladiumConverter = ((RecycledContentMetals * RICE_palladiumConverter) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_palladiumConverter))* SRmetals

RICE_recycling_platinumConverter = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_platinumConverter) - (RecycledContentMetals * RICE_platinumConverter)
RICE_primary_platinumConverter = ((RecycledContentMetals * RICE_platinumConverter) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_platinumConverter))* SRmetals

RICE_recycling_rhodiumConverter = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_rhodiumConverter) - (RecycledContentMetals * RICE_rhodiumConverter)
RICE_primary_rhodiumConverter = ((RecycledContentMetals * RICE_rhodiumConverter) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_rhodiumConverter))* SRmetals

RICE_recycling_chromiumsteelConverter = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_chromiumsteelConverter) - (RecycledContentMetals * RICE_chromiumsteelConverter)
RICE_primary_chromiumsteelConverter = ((RecycledContentMetals * RICE_chromiumsteelConverter) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_chromiumsteelConverter))* SRmetals

RICE_recycling_aluminiumEle = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_aluminiumEle) - (RecycledContentMetals * RICE_aluminiumEle)
RICE_primary_aluminiumEle = ((RecycledContentMetals * RICE_aluminiumEle) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_aluminiumEle))* SRmetals

RICE_recycling_copperEle = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_copperEle) - (RecycledContentMetals * RICE_copperEle)
RICE_primary_copperEle = ((RecycledContentMetals * RICE_copperEle) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_copperEle))* SRmetals

RICE_recycling_leadEle = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_leadEle) - (RecycledContentMetals * RICE_leadEle)
RICE_primary_leadEle = ((RecycledContentMetals * RICE_leadEle) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_leadEle))* SRmetals

RICE_recycling_nickelEle = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_nickelEle) - (RecycledContentMetals * RICE_nickelEle)
RICE_primary_nickelEle = ((RecycledContentMetals * RICE_nickelEle) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_nickelEle))* SRmetals

RICE_recycling_platinumEle = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_platinumEle) - (RecycledContentMetals * RICE_platinumEle)
RICE_primary_platinumEle = ((RecycledContentMetals * RICE_platinumEle) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_platinumEle))* SRmetals

RICE_recycling_HDPEEle = (RecyclingPEHD * RecyclingEfficiencyPlastics * RICE_HDPEEle) - (RecycledContentPlastic * RICE_HDPEEle)
RICE_primary_HDPEEle = ((RecycledContentPlastic * RICE_HDPEEle) - (RecyclingPEHD * RecyclingEfficiencyPlastics * RICE_HDPEEle)) * SRinert

RICE_recycling_reinforcingsteelEle = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelEle) - (RecycledContentMetals * RICE_reinforcingsteelEle)
RICE_primary_reinforcingsteelEle = ((RecycledContentMetals * RICE_reinforcingsteelEle) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelEle))* SRmetals

RICE_recycling_reinforcingsteelEle = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelEle) - (RecycledContentMetals * RICE_reinforcingsteelEle)
RICE_primary_reinforcingsteelEle = ((RecycledContentMetals * RICE_reinforcingsteelEle) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelEle))* SRmetals

RICE_recycling_zincEle = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_zincEle) - (RecycledContentMetals * RICE_chromiumsteelGasmotor)
RICE_primary_zincEle = ((RecycledContentMetals * RICE_zincEle) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_chromiumsteelGasmotor))* SRmetals

RICE_recycling_castironGen = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_castironGen) - (RecycledContentMetals * RICE_castironGen)
RICE_primary_castironGen = ((RecycledContentMetals * RICE_castironGen) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_castironGen))* SRmetals

RICE_recycling_copperGen = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_copperGen) - (RecycledContentMetals * RICE_copperGen)
RICE_primary_copperGen = ((RecycledContentMetals * RICE_copperGen) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_copperGen))* SRmetals

RICE_recycling_reinforcingsteelSan = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelSan) - (RecycledContentMetals * RICE_reinforcingsteelSan)
RICE_primary_reinforcingsteelSan = ((RecycledContentMetals * RICE_reinforcingsteelSan) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelSan))* SRmetals

RICE_recycling_aluminiumStorage = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_aluminiumStorage) - (RecycledContentMetals * RICE_aluminiumStorage)
RICE_primary_aluminiumStorage = ((RecycledContentMetals * RICE_aluminiumStorage) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_aluminiumStorage))* SRmetals

RICE_recycling_reinforcingsteelStorage = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelStorage) - (RecycledContentMetals * RICE_reinforcingsteelStorage)
RICE_primary_reinforcingsteelStorage = ((RecycledContentMetals * RICE_reinforcingsteelStorage) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_reinforcingsteelStorage))* SRmetals

RICE_recycling_sheetrollingaluminiumStorage = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_sheetrollingaluminiumStorage) - (RecycledContentMetals * RICE_sheetrollingaluminiumStorage)
RICE_primary_sheetrollingaluminiumStorage = ((RecycledContentMetals * RICE_sheetrollingaluminiumStorage) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_sheetrollingaluminiumStorage))* SRmetals

RICE_recycling_sheetrollingsteelStorage = (RecyclingMetals * RecyclingEfficiencyMetals * RICE_sheetrollingsteelStorage) - (RecycledContentMetals * RICE_sheetrollingsteelStorage)
RICE_primary_sheetrollingsteelStorage = ((RecycledContentMetals * RICE_sheetrollingsteelStorage) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_sheetrollingsteelStorage))* SRmetals

RICE_recycling_chromiumsteelStorage = (RecyclingMetals2 * RecyclingEfficiencyMetals * RICE_chromiumsteelStorage) - (RecycledContentMetals * RICE_chromiumsteelStorage)
RICE_primary_chromiumsteelStorage = ((RecycledContentMetals * RICE_chromiumsteelStorage) - (RecyclingMetals * RecyclingEfficiencyMetals * RICE_chromiumsteelStorage))* SRmetals


## 3.11 Module D3 (related to the export of energy as result of waste incineration)

In [18]:
RICE_energyrecoveryheat_HDPEControlcabinet =  - RICE_EOL_inc_HDPEControlcabinet * LHVHDPE * Xincheat
RICE_energyrecoveryele_HDPEControlcabinet =  - RICE_EOL_inc_HDPEControlcabinet * LHVHDPE * Xincele

RICE_energyrecoveryheat_PVCemulsionatedControlcabinet =  - RICE_EOL_inc_PVCemulsionatedControlcabinet * LHVPVC * Xincheat
RICE_energyrecoveryele_PVCemulsionatedControlcabinet =  - RICE_EOL_inc_PVCemulsionatedControlcabinet * LHVPVC * Xincele

RICE_energyrecoveryheat_PVCpolimerizedControlcabinet =  - RICE_EOL_inc_PVCpolimerizedControlcabinet * LHVPVC * Xincheat
RICE_energyrecoveryele_PVCpolimerizedControlcabinet =  - RICE_EOL_inc_PVCpolimerizedControlcabinet * LHVPVC * Xincele

RICE_energyrecoveryheat_corrugatedboxboardConverter =  - RICE_EOL_inc_corrugatedboxboardConverter * LHVCardboard * Xincheat
RICE_energyrecoveryele_corrugatedboxboardConverter =  - RICE_EOL_inc_corrugatedboxboardConverter * LHVCardboard * Xincele

RICE_energyrecoveryheat_HDPEEle =  - RICE_EOL_inc_HDPEEle * LHVHDPE * Xincheat
RICE_energyrecoveryele_HDPEEle =  - RICE_EOL_inc_HDPEEle * LHVHDPE * Xincele

RICE_energyrecoveryheat_PVCpolimerizedEle =  - RICE_EOL_inc_PVCpolimerizedEle * LHVPVC * Xincheat
RICE_energyrecoveryele_PVCpolimerizedEle =  - RICE_EOL_inc_PVCpolimerizedEle * LHVPVC * Xincele

RICE_energyrecoveryheat_alkydpaintStorage = - RICE_EOL_inc_alkydpaintStorage * LHVHazardous * Xincheat
RICE_energyrecoveryele_alkydpaintStorage = - RICE_EOL_inc_alkydpaintStorage * LHVHazardous * Xincele

RICE_energyrecoveryheat_zeoliteConverter = - RICE_EOL_inc_zeoliteConverter * LHVHazardous * Xincheat
RICE_energyrecoveryele_zeoliteConverter = - RICE_EOL_inc_zeoliteConverter * LHVHazardous * Xincele


# 4. Data for export

In [19]:
ThermalEnergyProduced_RICE = TotEnergyNeeds